In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

class Scraper():
    base_url = 'https://listado.mercadolibre.com.mx/'
    def scraping(self):
        product_name = input("\nProducto: ")
        cleaned_name = product_name.replace(" ", "-").lower()
        urls = [self.base_url + cleaned_name]

        page_number = 50
        for i in range(0, 10000, 50):
            urls.append(f"{self.base_url}{cleaned_name}_Desde_{page_number + 1}_NoIndex_True")
            page_number += 50

        self.data = []
        c = 1
            
        for i, url in enumerate(urls, start=1):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.find_all('li', class_='ui-search-layout__item')
            
            if not content:
                break

            for post in content:
                post_link = post.find("a")["href"]
                post_data = {
                    # "title": title,
               #     "price": price,
                    "post link": post_link,        
                }
                self.data.append(post_data)
                c += 1

    def export_to_csv(self):
        df = pd.DataFrame(self.data)
        df.to_csv(r"ML_data666.csv", sep=";")

if __name__ == "__main__":
    s = Scraper()
    s.scraping()
    s.export_to_csv()


df = pd.read_csv("ML_data666.csv")  
def extract_code(link):
    # extraer el código entre '/p/' y '#wid%'
    match = re.search(r'/p/([^#]*)#polycard', link)
    if match:
        return match.group(1)
    match_mlm = re.search(r'mx/MLM-(\d+)-', link)
    if match_mlm:
        return f"MLM{match_mlm.group(1)}"
    return None

def create_new_link(code):
    if code:
        return f"https://www.mercadolibre.com.mx/noindex/catalog/reviews/{code}?noIndex=true&access=view_all&modal=true&controlled=true"
    return None

df['CodigoExtraido'] = df[';post link'].apply(extract_code)
df['NuevoLink'] = df['CodigoExtraido'].apply(create_new_link)
df_final = df[['NuevoLink']].dropna()
df_final.to_csv("ML_data666.csv", index=False)


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

df_urls = pd.read_csv("ML_data666.csv")
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
all_reviews = []

def scrape_url(reviews_url):
    reviews_texts = []
    try:
        response = requests.get(reviews_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            reviews = soup.find_all("p", {"class": "ui-review-capability-comments__comment__content ui-review-capability-comments__comment__content"})  # Verifica la clase correcta
            reviews_texts = [review.get_text().strip() for review in reviews]
        else:
            print(f"No se pudo acceder a la página de reseñas para la URL: {reviews_url}")
    except Exception as e:
        print(f"Error al procesar la URL {reviews_url}: {e}")
    return reviews_texts

with ThreadPoolExecutor(max_workers=7) as executor:
    future_to_url = {executor.submit(scrape_url, url): url for url in df_urls.iloc[:, 0]}
    for future in as_completed(future_to_url):
        reviews = future.result()
        all_reviews.extend(reviews)

df_reviews = pd.DataFrame(all_reviews, columns=["Reseña"])
df_reviews.to_csv("reseñas_ML_data666.csv", index=False, encoding='utf-8')

